# New York Times Annotated Corpus keyword labeling with pre-trained word embeddings

In this notebook, we'll use pre-trained [GloVe word embeddings](http://nlp.stanford.edu/projects/glove/) for keyword labeling using Keras (version $\ge$ 2 is required). This notebook is largely based on the blog post [Using pre-trained word embeddings in a Keras model](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) by François Chollet.

**Note that using a GPU with this notebook is highly recommended.**

First, the needed imports. Keras tells us which backend (Theano, Tensorflow, CNTK) it will be using.

In [ ]:
%matplotlib inline

from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import LSTM, CuDNNLSTM
from keras.utils import to_categorical

from distutils.version import LooseVersion as LV
from keras import __version__
from keras import backend as K

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import train_test_split
from sklearn import metrics

import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

If we are using TensorFlow as the backend, we can use TensorBoard to visualize our progress during training.

In [ ]:
if K.backend() == "tensorflow":
    import tensorflow as tf
    print('TensorFlow version:', tf.__version__)
    from keras.callbacks import TensorBoard
    import os, datetime
    logdir = os.path.join(os.getcwd(), "logs",
                     "ted-"+datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    print('TensorBoard log directory:', logdir)
    os.makedirs(logdir)
    callbacks = [TensorBoard(log_dir=logdir)]
else:
    callbacks =  None

## GloVe word embeddings

Let's begin by loading a datafile containing pre-trained word embeddings.  The datafile contains 100-dimensional embeddings for 400,000 English words.  

In [ ]:
#!wget --content-disposition -nc https://kannu.csc.fi/s/rrCNCRdJf9LZSCE/download
GLOVE_DIR = "/home/cloud-user/machine-learning-scripts/notebooks"

#GLOVE_DIR = "/home/cloud-user/glove.6B"

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_dim = len(coefs)
print('Found %d word vectors of dimensionality %d.' % (len(embeddings_index), embedding_dim))

print('Examples of embeddings:')
for w in ['some', 'random', 'words']:
    print(w, embeddings_index[w])

## New York Times Annotated Corpus

Next we'll load the [New York Times Annotated Corpus](https://catalog.ldc.upenn.edu/ldc2008t19).  The data is originally in stored in article-wise XML files, but we load a portion of the data from a preprocessed HDF5 file instead. 

The preprocessed dataset contains articles of year 1987. Each article is annotated with a set of tags.

In [ ]:
TEXT_DATA_DIR = "/home/cloud-user/nytac"

store = pd.HDFStore(TEXT_DATA_DIR+'/1987.h5')
df = store['df']
labels_all_sorted = store['labels_all_sorted']
store.close()

print(len(df), 'articles')
df.sample(10)

### Textual data

There are two potential columns to be used as the input text source: `full_text` and `lead_paragraph`. The former is the full text of the article, whereas the latter is a shorter abstract of the contents of the article. 

Let's inspect the distributions of the lengths of these columns: 

In [ ]:
len_lead, len_full = np.empty(len(df)), np.empty(len(df))
for i, row in df.iterrows():
   len_lead[i]=len(row['lead_paragraph'])
   len_full[i]=len(row['full_text'])

plt.figure(figsize=(15,5))
plt.subplot(121)
plt.title('Length of lead_paragraphs, mean: %.2f' % np.mean(len_lead))
plt.xlabel('words')
plt.hist(len_lead, 'auto')
plt.subplot(122)
plt.title('Length of full_text, mean: %.2f' % np.mean(len_full))
plt.xlabel('words')
plt.hist(len_full, 'auto');

Let's take a look at the lead paragraphs and full texts of 5 random articles:

In [ ]:
dfs = df.sample(5)
for i, row in dfs.iterrows():
    print('='*80)
    print (i, ': ', row['title'], '\n\n', row['lead_paragraph'], '\n\n', row['full_text'], sep="")

Now we decide to use either the `transcipt` or the `description` column:

In [ ]:
texttype = "full_text"
#texttype = "lead_paragraph"

### Keywords

We use the `NLABELS` most frequent descriptor tags as keyword labels we wish to predict. We'll encode the labels as `NLABELS`-dimensional binary vectors and store these in a new column `labels`.

Then we take a look at the most common tags, and plot histograms of all tags and the used labels.

In [ ]:
NLABELS=100

def indices_to_labels(x):
    labels = np.zeros(NLABELS)
    for i in x:
        if i < NLABELS:
            labels[i] = 1
    return labels

df['labels'] = df['descriptor_indices'].apply(indices_to_labels)

ntags = dict()
l, ll = np.zeros(len(df)), np.zeros(len(df))

for i, tl in df['descriptor'].iteritems():    
    for t in tl:
        if t == "Terms not available":
            continue
        l[i] += 1
        if t in ntags:
            ntags[t] += 1
        else:
            ntags[t] = 1

for i, labv in df['labels'].iteritems():    
    ll[i] = np.sum(labv)
            
nrows = len(df)
print('Total of', len(labels_all_sorted), 'descriptor tags. Showing', NLABELS, 'most common tags:')
for i, t in enumerate(labels_all_sorted[:NLABELS]):
    print(i, t, ntags[t], "%.4f" % (ntags[t]/nrows))

plt.figure(figsize=(10,5))
plt.title('All descriptor tags and used labels, means: %.2f tags, %.2f labels' % (np.mean(l), np.mean(ll)))
plt.xlabel('tags/labels per article')
plt.hist([l,ll],np.arange(0,15), align='left', label=['descriptor tags', 'labels'])
plt.legend(loc='best');

### Produce input and label tensors

We vectorize the text samples and labels into a 2D integer tensors. `MAX_NUM_WORDS` is the number of different words to use as tokens, selected based on word frequency. `MAX_SEQUENCE_LENGTH` is the fixed sequence length obtained by truncating or padding the original sequences. 

In [ ]:
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000 

tokenizer = text.Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts([x for x in df[texttype]])
sequences = tokenizer.texts_to_sequences([x for x in df[texttype]])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.asarray([x for x in df['labels']])

print('Shape of data tensor:', data.shape)
print('Shape of labels tensor:', labels.shape)

Next, we split the data into a training, validation, and test sets.

In [ ]:
VALIDATION_SET, TEST_SET = 10000, 10000

x_train, x_test, y_train, y_test = train_test_split(data, labels, 
                                                    test_size=TEST_SET,
                                                    shuffle=True, random_state=42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size=VALIDATION_SET,
                                                  shuffle=False)

print('Shape of training data tensor:', x_train.shape)
print('Shape of training label tensor:', y_train.shape)
print('Shape of validation data tensor:', x_val.shape)
print('Shape of validation label tensor:', y_val.shape)
print('Shape of test data tensor:', x_test.shape)
print('Shape of test label tensor:', y_test.shape)

We prepare the embedding matrix by retrieving the corresponding word embedding for each token in our vocabulary:

In [ ]:
print('Preparing embedding matrix.')

num_words = min(MAX_NUM_WORDS, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print('Shape of embedding matrix:', embedding_matrix.shape)

## 1-D CNN

### Initialization


In [ ]:
print('Build model...')
model = Sequential()

model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
#model.add(Dropout(0.2))

model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dense(NLABELS, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')

print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

### Learning

In [ ]:
%%time
epochs = 10
batch_size=64

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    verbose=2, callbacks=callbacks)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'], label='training')
plt.plot(history.epoch,history.history['val_loss'], label='validation')
plt.title('loss')
plt.legend(loc='best');

### Inference

To further analyze the results, we can produce the actual predictions for the test data.

In [ ]:
predictions = model.predict(x_test)

The selected threshold controls the number of label predictions we'll make:

In [ ]:
threshold = 0.25

avg_n_gt, avg_n_pred = 0, 0
for t in range(len(y_test)):
    avg_n_gt += len(np.where(y_test[t]>0.5)[0])
    avg_n_pred += len(np.where(predictions[t]>threshold)[0])
avg_n_gt /= len(y_test)
avg_n_pred /= len(y_test)
print('Average number of ground-truth labels per talk: %.2f' % avg_n_gt)
print('Average number of predicted labels per talk: %.2f' % avg_n_pred)

Let's look at the correct and predicted labels for some talks in the validation set.

In [ ]:
nb_articles_to_show = 20

for t in range(nb_articles_to_show):
    print(t,':')
    print('    correct: ', end='')
    for idx in np.where(y_test[t]>0.5)[0].tolist():
        sys.stdout.write('['+labels_all_sorted[idx]+'] ')
    print()
    print('  predicted: ', end='')
    for idx in np.where(predictions[t]>threshold)[0].tolist():
        sys.stdout.write('['+labels_all_sorted[idx]+'] ')
    print()

Scikit-learn has some applicable performance [metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) we can try: 

In [ ]:
print('Precision: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.precision_score(y_test.flatten(), predictions.flatten()>threshold), threshold))
print('Recall: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.recall_score(y_test.flatten(), predictions.flatten()>threshold), threshold))
print('F1 score: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.f1_score(y_test.flatten(), predictions.flatten()>threshold), threshold))

average_precision = metrics.average_precision_score(y_test.flatten(), predictions.flatten())
print('Average precision: {0:.3f}'.format(average_precision))
print('Coverage: {0:.3f}'
      .format(metrics.coverage_error(y_test, predictions)))
print('LRAP: {0:.3f}'
      .format(metrics.label_ranking_average_precision_score(y_test, predictions)))

precision, recall, _ = metrics.precision_recall_curve(y_test.flatten(), predictions.flatten())
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-recall curve');

## LSTM

### Initialization

In [ ]:
print('Build model...')
model = Sequential()

model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
#model.add(Dropout(0.2))

model.add(CuDNNLSTM(256, return_sequences=True))
model.add(CuDNNLSTM(256))

model.add(Dense(128, activation='relu'))
model.add(Dense(NLABELS, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')

print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

### Learning

In [ ]:
%%time
epochs = 10
batch_size=64

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    verbose=2, callbacks=callbacks)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'], label='training')
plt.plot(history.epoch,history.history['val_loss'], label='validation')
plt.title('loss')
plt.legend(loc='best');

### Inference

In [ ]:
predictions = model.predict(x_test)

In [ ]:
threshold = 0.5

avg_n_gt, avg_n_pred = 0, 0
for t in range(len(y_test)):
    avg_n_gt += len(np.where(y_test[t]>0.5)[0])
    avg_n_pred += len(np.where(predictions[t]>threshold)[0])
avg_n_gt /= len(y_test)
avg_n_pred /= len(y_test)
print('Average number of ground-truth labels per talk: %.2f' % avg_n_gt)
print('Average number of predicted labels per talk: %.2f' % avg_n_pred)

In [ ]:
nb_articles_to_show = 20

for t in range(nb_articles_to_show):
    print(t,':')
    print('    correct: ', end='')
    for idx in np.where(y_test[t]>0.5)[0].tolist():
        sys.stdout.write('['+labels_all_sorted[idx]+'] ')
    print()
    print('  predicted: ', end='')
    for idx in np.where(predictions[t]>threshold)[0].tolist():
        sys.stdout.write('['+labels_all_sorted[idx]+'] ')
    print()

In [ ]:
print('Precision: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.precision_score(y_test.flatten(), predictions.flatten()>threshold), threshold))
print('Recall: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.recall_score(y_test.flatten(), predictions.flatten()>threshold), threshold))
print('F1 score: {0:.3f} (threshold: {1:.2f})'
      .format(metrics.f1_score(y_test.flatten(), predictions.flatten()>threshold), threshold))

average_precision = metrics.average_precision_score(y_test.flatten(), predictions.flatten())
print('Average precision: {0:.3f}'.format(average_precision))
print('Coverage: {0:.3f}'
      .format(metrics.coverage_error(y_test, predictions)))
print('LRAP: {0:.3f}'
      .format(metrics.label_ranking_average_precision_score(y_test, predictions)))

precision, recall, _ = metrics.precision_recall_curve(y_test.flatten(), predictions.flatten())
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-recall curve');